In [1]:
import asyncio
import json
import re
from urllib.request import Request, urlopen

from bs4 import BeautifulSoup
import pandas as pd

In [2]:
raw_pv_df = pd.read_csv('page_views.csv')
raw_pv_df.head()

f = open("data.json", "r")
txt = f.read()

def removeNonAscii(s):
    return "".join(i for i in s if ord(i)<128)

data = json.loads(removeNonAscii(txt))
print(str(data)[:500] + ' ...')

{'rss': {'channel': {'title': 'Daily Bruin', 'link': 'https://dailybruin.com', 'description': 'Since 1919', 'pubDate': 'Tue, 03 Nov 2020 23:28:02 +0000', 'language': 'en-US', 'wxr_version': {'__text': '1.2'}, 'base_site_url': {'__text': 'https://dailybruin.com'}, 'base_blog_url': {'__text': 'https://dailybruin.com'}, 'author': [{'author_id': {'__text': '6265'}, 'author_login': {'__cdata': 'Camayak_43fdcc6551'}, 'author_email': {'__cdata': 'jshapero@dailybruin.com'}, 'author_display_name': {'__cd ...


In [3]:
raw_xml_df = pd.DataFrame.from_dict(data['rss']['channel']['item']).drop(columns = ["comment_status", "description", "guid", "is_sticky", "menu_order", "ping_status", "post_parent", "post_password", "post_id", "post_type", "postmeta", "status", "post_name", "post_date", "post_date_gmt"])
raw_xml_df.head()

,title,link,pubDate,creator,encoded,category
0,Revised California budget proposal to cut UC f...,https://dailybruin.com/2020/06/01/revised-cali...,"Mon, 01 Jun 2020 20:16:35 +0000",{'__cdata': 'Camayak_43fdcc6551'},[{'__cdata': '<p>The University of California ...,"[{'_domain': 'author', '_nicename': 'cap-julia..."
1,LCC Theatre Company embraces online format wit...,https://dailybruin.com/2020/06/01/lcc-theatre-...,"Mon, 01 Jun 2020 21:21:39 +0000",{'__cdata': 'Camayak_816'},"[{'__cdata': '""Lapus Pocket Theatre"" isnt a ph...","[{'_domain': 'author', '_nicename': 'cap-olivi..."
2,Q&A: UCLA baseball coach talks impact of short...,https://dailybruin.com/2020/06/01/qa-ucla-base...,"Mon, 01 Jun 2020 21:22:09 +0000",{'__cdata': 'Camayak_a72e6b9872'},"[{'__cdata': '<i>Almost one year ago, the 2019...","[{'_domain': 'author', '_nicename': 'cap-jared..."
3,"Coaches, athletes turn to social media to expr...",https://dailybruin.com/2020/06/01/coaches-athl...,"Mon, 01 Jun 2020 22:18:39 +0000",{'__cdata': 'Camayak_6cbf65a89c'},[{'__cdata': 'Since the death of George Floyd ...,"[{'_domain': 'author', '_nicename': 'cap-sam-c..."
4,USAC leadership requests immediate removal of ...,https://dailybruin.com/2020/06/01/usac-leaders...,"Tue, 02 Jun 2020 00:29:02 +0000",{'__cdata': 'Camayak_55924fa9bd'},[{'__cdata': '<p>Student government is calling...,"[{'_domain': 'author', '_nicename': 'cap-berna..."


In [4]:
def parse_categories(items):
    authors = []
    categories = []
    for d in items:
        if d['_domain'] == 'author':
            authors.append(d['_nicename'])
        if d['_domain'] == 'category':
            categories.append(d['_nicename'])
    return authors, categories


def parse_db_url_path(url):
    regex = r"https://(?:.*.)?dailybruin.com(\/.*)"
    matches = re.match(regex, url)
    if matches is None:
        raise ValueError(f"[parse_db_url_path] - Invalid URL - {url}")
    return matches.group(1)


xml_df = raw_xml_df
xml_df.creator = xml_df.creator.map(lambda x: x['__cdata'])
xml_df.encoded = xml_df.encoded.map(lambda x: x[0]['__cdata'])

authors_and_categories = xml_df.category.map(parse_categories)
xml_df['authors'] = [a for a, _ in authors_and_categories]
xml_df['categories'] = [c for _, c in authors_and_categories]

xml_df['page'] = xml_df.link.map(parse_db_url_path)

xml_df = xml_df.drop('category', axis=1)
xml_df = xml_df.rename(columns={'encoded': 'encoded_text'})

xml_df.head()

,title,link,pubDate,creator,encoded_text,authors,categories,page
0,Revised California budget proposal to cut UC f...,https://dailybruin.com/2020/06/01/revised-cali...,"Mon, 01 Jun 2020 20:16:35 +0000",Camayak_43fdcc6551,<p>The University of California could lose $37...,[cap-julia-shapero],"[higher-education, news, university-of-califor...",/2020/06/01/revised-california-budget-proposal...
1,LCC Theatre Company embraces online format wit...,https://dailybruin.com/2020/06/01/lcc-theatre-...,"Mon, 01 Jun 2020 21:21:39 +0000",Camayak_816,"""Lapus Pocket Theatre"" isnt a physical venue ...",[cap-olivia-mazzucato],"[arts-entertainment, film-tv]",/2020/06/01/lcc-theatre-company-embraces-onlin...
2,Q&A: UCLA baseball coach talks impact of short...,https://dailybruin.com/2020/06/01/qa-ucla-base...,"Mon, 01 Jun 2020 21:22:09 +0000",Camayak_a72e6b9872,"<i>Almost one year ago, the 2019 MLB Draft con...",[cap-jared-tay],"[baseball, sports]",/2020/06/01/qa-ucla-baseball-coach-talks-impac...
3,"Coaches, athletes turn to social media to expr...",https://dailybruin.com/2020/06/01/coaches-athl...,"Mon, 01 Jun 2020 22:18:39 +0000",Camayak_6cbf65a89c,"Since the death of George Floyd on May 25, pro...",[cap-sam-connon],"[athletics-news, sports]",/2020/06/01/coaches-athletes-turn-to-social-me...
4,USAC leadership requests immediate removal of ...,https://dailybruin.com/2020/06/01/usac-leaders...,"Tue, 02 Jun 2020 00:29:02 +0000",Camayak_55924fa9bd,<p>Student government is calling on the state ...,[cap-bernard-mendez],"[california, campus, los-angeles, national, news]",/2020/06/01/usac-leadership-requests-immediate...


In [5]:
def sanitize_page(page):
    print(page)
    if "?" in page:
        page = page.split("?")[0]
    if page[0] != "/":
        page = "/" + page
    if page[-1] != "/":
        page += '/'
    return page


pv_df = raw_pv_df.rename(
    columns={
        'Page': 'page',
        'Pageviews': 'views',
        'Unique Pageviews': 'unique_views',
        'Avg. Time on Page': 'avg_time',
        'Entrances': 'entrances',
        'Bounce Rate': 'bounce_rate',
        '% Exit': 'exit_percent',
        'Page Value': 'page_value',
    }
)


pv_df = pv_df[pv_df['page'].notna()]
pv_df.page = pv_df.page.map(sanitize_page)

pv_df.head()

/
/2020/06/04/students-sign-petition-to-fire-ucla-lecturer-after-many-find-his-email-insensitive
/2020/07/11/album-review-juice-wrld-proves-legends-never-die-in-album-released-after-his-death
/category/news
/2020/10/24/student-dies-after-falling-from-westwood-apartment-building
/2019/04/12/as-sesquicentennial-occurs-debate-on-arrangement-of-periodic-table-continues
/category/sports
/2020/07/16/ucs-2020-acceptance-rate-rose-to-69-5-admitting-record-number-of-students
/2020/07/05/ucla-school-of-dentistry-fails-to-implement-many-covid-19-mitigating-practices
/2020/09/18/landlord-arrested-for-vandalism-in-attempt-to-evict-ucla-students-before-end-of-lease
/2020/06/15/ucla-announces-fall-quarter-plans-for-classes-housing-health-precautions
/2010/10/21/pros_and_cons_of_prop_23
/2019/09/19/getting-into-ucla-is-hard-getting-into-these-ucla-schools-is-harder
/2020/06/27/under-armour-seeks-to-end-record-breaking-sponsorship-deal-with-ucla
/2020/07/05/the-blm-glossary-hashtags-and-terms-to-know
/

/2019/10/19/the-quad-an-overview-of-fires-near-ucla-in-recent-years-and-their-causes
/2020/05/06/student-seeks-to-spread-aloha-spirit-in-hawaiian-inspired-line-of-wire-jewelry
/2020/05/11/qa-sophia-james-discusses-experiences-musical-journey-on-american-idol
/2020/06/10/film-program-projects-new-showcase-plans-postpones-some-amid-pandemic
/2020/06/18/quarantine-diaries-international-students
/2020/08/28/movie-review-get-duked-falls-flat-with-one-dimensional-characters-lacks-laugh-out-loud-moments
/2020/10/28/opinion-ucla-must-suspend-classes-on-election-day-to-give-students-time-to-vote
/2014/08/04/graphic-equivalents-of-20-million-gallons-of-water
/2014/10/31/thread-count-revisiting-ucla-footballs-alternate-uniforms
/2018/10/25/on-campus-health-care-should-be-option-for-all-students-not-just-those-with-uc-ship
/2020/01/29/anti-hazing-advocate-speaks-at-ucla-pushing-for-students-to-speak-up
/2020/02/20/ucla-experts-dispute-effectiveness-of-airport-screening-in-detecting-coronavirus
/20

/2020/10/16/ucla-faculty-students-say-pandemic-may-alter-way-people-vote-in-2020-and-beyond
/2020/10/24/student-dies-after-falling-from-westwood-apartment-building?from=groupmessage
/author/cecile-wu
/category/theater-arts
/1996/04/10/ucla-student-found-dead-in-dor
/2011/09/12/faith_illuminated
/2012/11/02/womens-volleyball-seeks-wins-at-away-games
/2014/12/15/new-amazon-lockers-provide-students-package-delivery-alternative
/2016/11/06/the-quad-big-fat-fatty-a-not-so-easy-solution-to-greasy-cravings
/2018/11/28/campus-queries-what-is-the-impact-of-wildfires-on-wildlife-in-california
/2019/02/18/second-take-failure-of-ajita-battle-angel-shows-that-costly-cgi-cant-save-a-bad-film
/2019/05/28/students-voice-concerns-about-bathroom-policy-security-camera-monitoring
/2019/10/23/jorge-salcedo-charged-with-fraud-and-bribery-in-ongoing-admissions-scandal
/2019/10/23/lsat-to-be-made-more-accessible-for-visually-impaired-students-after-lawsuit
/2019/12/04/ucla-must-use-its-resources-to-amend-col

/2018/09/26/concert-review-florence-and-the-machines-recent-tour-introduces-a-more-stripped-down-aesthetic
/2018/09/26/enormous-activities-fair-limits-search-for-clubs-to-zero-week-on-campus-students
/2018/10/18/submission-new-security-camera-policy-could-create-more-problems-than-it-solves
/2018/10/24/event-features-north-korean-refugees-discussing-personal-stories-experiences
/2018/11/01/movie-review-bohemian-rhapsody-succeeds-as-a-triumphant-celebration-of-freddie-mercurys-life
/2018/11/12/dashew-center-fails-to-provide-adequate-efficient-support-for-international-students
/2018/12/08/ucla-researchers-attempt-to-understand-tusklessness-among-mozambique-elephants
/2019/02/27/owner-of-tlt-food-speaks-on-success-of-restaurant-goals-for-ucla-community
/2019/03/10/gallery-ucla-gymnastics-flies-to-victory-over-stanford
/2019/04/03/art-to-heart-students-reinterpret-big-bertha-to-subvert-art-worlds-miasma-of-elitism
/2019/04/04/asu-continues-long-standing-push-for-resources-dedicated-to-bla

/2009/10/13/security-cameras-monitor-campus
/2010/05/24/system-declare-degree-candidacy-unfairly-slams-stu
/2010/11/18/like_other_art_forms_video_games_shouldnt_be_censored_by_the_law
/2011/02/22/the_annual_palestine_awareness_week_kicks_off_today_with_a_series_of_events_and_speakers_to_educate_
/2012/02/22/president_obama_awards_history_professor_teofilo_ruiz_the_national_humanities_medal
/2012/11/05/ucla-professor-recreates-religious-images-in-art
/2013/11/08/qa-sports-history-professor-talks-about-uclas-john-wooden
/2013/11/14/ucla-soccer-senior-reed-williams-looks-forward-to-shaking-off-slump
/2013/11/25/feature-head-terry-gray-releases-his-first-rap-mixtape-american-paradox
/2014/02/10/ursa-other-ucla-sites-may-cease-in-spring-with-myucla-integration
/2014/03/03/former-ucla-water-polo-player-faces-sexual-assault-charges
/2014/06/02/out-of-focus-brilliance-of-days-of-heaven-lies-in-filmmakers-reliance-on-visuals
/2014/08/18/former-student-counselor-lovell-sevilla-dies-from-leukemia

,page,views,unique_views,avg_time,entrances,bounce_rate,exit_percent,page_value
0,/,"90,941","74,000",0:01:24,"63,825",45.99%,45.02%,$0.00
1,/2020/06/04/students-sign-petition-to-fire-ucl...,"41,497","38,105",0:04:55,"37,002",90.70%,89.16%,$0.00
2,/2020/07/11/album-review-juice-wrld-proves-leg...,"24,298","18,392",0:02:35,"18,345",79.60%,75.12%,$0.00
3,/category/news/,"20,575","15,296",0:01:23,"5,014",47.06%,33.24%,$0.00
4,/2020/10/24/student-dies-after-falling-from-we...,"20,347","17,904",0:04:06,"15,854",85.16%,82.16%,$0.00


In [6]:
def parse_percent(s):
    return float(str(s).strip('%')) / 100


def parse_float(s):
    return float(str(s).replace(",", ""))


def parse_seconds(time):
    if str(time) == 'nan' or str(time) == 'None':
        return ''
    mins = int(str(time)[2:4])
    second = int(str(time)[5:])
    total_seconds = mins * 60 + second
    return total_seconds


def clean_html(html):
    html = re.sub(r'<.*?>', ' ', str(html))
    html = re.sub(r'\n', ' ', html)
    html = re.sub(r'\\', ' ', html)
    return html

def article_len(article):
    return(len(article))



df = pd.merge(
    pv_df, xml_df, how="outer", left_on="page", right_on="page"
)

df.exit_percent = df.exit_percent.map(parse_percent)
df.bounce_rate = df.bounce_rate.map(parse_percent)
df.views = df.views.map(parse_float)
df.unique_views = df.unique_views.map(parse_float)
df.entrances = df.entrances.map(parse_float)
# df.avg_time = df.avg_time.map(parse_seconds)
df.encoded_text = df.encoded_text.map(clean_html)

df.head()

,page,views,unique_views,avg_time,entrances,bounce_rate,exit_percent,page_value,title,link,pubDate,creator,encoded_text,authors,categories
0,/,90941.0,74000.0,0:01:24,63825.0,0.4599,0.4502,$0.00,Graduation Issue 2020,https://gradissue2020.dailybruin.com/,"Fri, 12 Jun 2020 17:09:01 +0000",Camayak_180,We are past the tipping point. Where do we go ...,[camayak_180],[interactive-page]
1,/2020/06/04/students-sign-petition-to-fire-ucl...,41497.0,38105.0,0:04:55,37002.0,0.9070,0.8916,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]"
2,/2020/06/04/students-sign-petition-to-fire-ucl...,463.0,395.0,0:04:50,391.0,0.8670,0.8488,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]"
3,/2020/06/04/students-sign-petition-to-fire-ucl...,179.0,162.0,0:03:50,153.0,0.7647,0.7542,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]"
4,/2020/06/04/students-sign-petition-to-fire-ucl...,92.0,79.0,0:05:28,75.0,0.8400,0.8370,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]"


In [7]:
lengths = []

for article in df['encoded_text']:
    lengths.append(len(article))
    
df['article_lens'] = lengths

df.head()

,page,views,unique_views,avg_time,entrances,bounce_rate,exit_percent,page_value,title,link,pubDate,creator,encoded_text,authors,categories,article_lens
0,/,90941.0,74000.0,0:01:24,63825.0,0.4599,0.4502,$0.00,Graduation Issue 2020,https://gradissue2020.dailybruin.com/,"Fri, 12 Jun 2020 17:09:01 +0000",Camayak_180,We are past the tipping point. Where do we go ...,[camayak_180],[interactive-page],108
1,/2020/06/04/students-sign-petition-to-fire-ucl...,41497.0,38105.0,0:04:55,37002.0,0.9070,0.8916,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]",5001
2,/2020/06/04/students-sign-petition-to-fire-ucl...,463.0,395.0,0:04:50,391.0,0.8670,0.8488,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]",5001
3,/2020/06/04/students-sign-petition-to-fire-ucl...,179.0,162.0,0:03:50,153.0,0.7647,0.7542,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]",5001
4,/2020/06/04/students-sign-petition-to-fire-ucl...,92.0,79.0,0:05:28,75.0,0.8400,0.8370,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]",5001


In [8]:
df = df.drop_duplicates(subset=['page']).dropna()
df = df.reset_index(drop=True)
df['image_url'] = None

df.head()

,page,views,unique_views,avg_time,entrances,bounce_rate,exit_percent,page_value,title,link,pubDate,creator,encoded_text,authors,categories,article_lens,image_url
0,/,90941.0,74000.0,0:01:24,63825.0,0.4599,0.4502,$0.00,Graduation Issue 2020,https://gradissue2020.dailybruin.com/,"Fri, 12 Jun 2020 17:09:01 +0000",Camayak_180,We are past the tipping point. Where do we go ...,[camayak_180],[interactive-page],108,None
1,/2020/06/04/students-sign-petition-to-fire-ucl...,41497.0,38105.0,0:04:55,37002.0,0.9070,0.8916,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,[cap-kari-lau],"[campus, news]",5001,None
2,/2020/07/11/album-review-juice-wrld-proves-leg...,24298.0,18392.0,0:02:35,18345.0,0.7960,0.7512,$0.00,Album review: Juice WRLD proves 'Legends Never...,https://dailybruin.com/2020/07/11/album-review...,"Sat, 11 Jul 2020 23:00:54 +0000",Camayak_2fac78eb7b,Emo-rap artist Juice WRLDs first posthumous al...,[cap-kanishka-mehra],"[arts-entertainment, ae-columns, music]",4810,None
3,/2020/10/24/student-dies-after-falling-from-we...,20347.0,17904.0,0:04:06,15854.0,0.8516,0.8216,$0.00,Student dies after falling from Westwood apart...,https://dailybruin.com/2020/10/24/student-dies...,"Sat, 24 Oct 2020 21:23:40 +0000",Camayak_351705866e,A student died early Saturday morning after f...,"[cap-elizabeth-sherwood, cap-bernard-mendez]","[news, westwood]",1826,None
4,/2020/07/16/ucs-2020-acceptance-rate-rose-to-6...,10921.0,10367.0,0:03:30,10004.0,0.9128,0.8958,$0.00,"UC's 2020 acceptance rate rose to 69.5%, admit...",https://dailybruin.com/2020/07/16/ucs-2020-acc...,"Fri, 17 Jul 2020 00:56:09 +0000",Camayak_bef6dcd253,This post was updated July 16 at 8:18 p.m. ...,[cap-genesis-qu],"[campus, news, university-of-california]",1369,None


In [9]:
def get_image_url(html):
    soup = BeautifulSoup(html)
    images = soup.findAll('meta', {'property': 'og:image'})
    if len(images) == 1:
        image = images[0]['content']
        return image
    else:
        print(f'Can\'t find image: {row.link}')

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
}

batch_size = 100

# Loops through batches of batch_size pages and scrapes images for them asynchronously
for i in range(0, df.shape[0], batch_size):
    print(f'Processing batch {i // batch_size} of {df.shape[0] // batch_size}')
    subset = df.iloc[i:i + batch_size]

    loop = asyncio.get_event_loop()
    requests = {
        'indices': [],
        'futures': [],
    }

    for j, row in subset.iterrows():
        print(j)
        if row.image_url is None:
            req = Request(row.link, headers=headers)
            future = loop.run_in_executor(None, urlopen, req)
            requests['indices'].append(j)
            requests['futures'].append(future)

    for j, res in zip(requests['indices'], await asyncio.gather(*requests['futures'])):
        html = res.read()
        df.image_url[j] = get_image_url(html)

Processing batch 0 of 7
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


HTTPError: HTTP Error 500: Internal Server Error

In [10]:
df.to_csv("merged.csv")

In [11]:
final = pd.read_csv("merged.csv")

def convertPercent(percent):
    return float(str(percent).strip('%'))/100

def percentToFloat(name):
    arr = final[name].to_numpy()
    new = []
    for value in arr:
        new.append(convertPercent(value))
    final[name] = new

percentToFloat("bounce_rate")
percentToFloat("exit_percent")

final.head()


,Unnamed: 0,page,views,unique_views,avg_time,entrances,bounce_rate,exit_percent,page_value,title,link,pubDate,creator,encoded_text,authors,categories,article_lens,image_url
0,0,/,90941.0,74000.0,0:01:24,63825.0,0.004599,0.004502,$0.00,Graduation Issue 2020,https://gradissue2020.dailybruin.com/,"Fri, 12 Jun 2020 17:09:01 +0000",Camayak_180,We are past the tipping point. Where do we go ...,['camayak_180'],['interactive-page'],108,NaN
1,1,/2020/06/04/students-sign-petition-to-fire-ucl...,41497.0,38105.0,0:04:55,37002.0,0.009070,0.008916,$0.00,Students sign petition to fire UCLA lecturer a...,https://dailybruin.com/2020/06/04/students-sig...,"Thu, 04 Jun 2020 23:47:55 +0000",Camayak_8ce8660ac1,This post was updated June 5 at 1:35 p.m. ...,['cap-kari-lau'],"['campus', 'news']",5001,NaN
2,2,/2020/07/11/album-review-juice-wrld-proves-leg...,24298.0,18392.0,0:02:35,18345.0,0.007960,0.007512,$0.00,Album review: Juice WRLD proves 'Legends Never...,https://dailybruin.com/2020/07/11/album-review...,"Sat, 11 Jul 2020 23:00:54 +0000",Camayak_2fac78eb7b,Emo-rap artist Juice WRLDs first posthumous al...,['cap-kanishka-mehra'],"['arts-entertainment', 'ae-columns', 'music']",4810,NaN
3,3,/2020/10/24/student-dies-after-falling-from-we...,20347.0,17904.0,0:04:06,15854.0,0.008516,0.008216,$0.00,Student dies after falling from Westwood apart...,https://dailybruin.com/2020/10/24/student-dies...,"Sat, 24 Oct 2020 21:23:40 +0000",Camayak_351705866e,A student died early Saturday morning after f...,"['cap-elizabeth-sherwood', 'cap-bernard-mendez']","['news', 'westwood']",1826,NaN
4,4,/2020/07/16/ucs-2020-acceptance-rate-rose-to-6...,10921.0,10367.0,0:03:30,10004.0,0.009128,0.008958,$0.00,"UC's 2020 acceptance rate rose to 69.5%, admit...",https://dailybruin.com/2020/07/16/ucs-2020-acc...,"Fri, 17 Jul 2020 00:56:09 +0000",Camayak_bef6dcd253,This post was updated July 16 at 8:18 p.m. ...,['cap-genesis-qu'],"['campus', 'news', 'university-of-california']",1369,NaN


In [ ]:
print(final.loc[[7]].to_numpy())

In [12]:
link = final.columns.get_loc("link")
views = final.columns.get_loc("views")
unique_views = final.columns.get_loc("unique_views")
entrances = final.columns.get_loc("entrances")
avg_time = final.columns.get_loc("avg_time")
bounce_rate = final.columns.get_loc("bounce_rate")
exit_percent = final.columns.get_loc("exit_percent")
article_lens = final.columns.get_loc("article_lens")



def combine(one, two):
    one.reset_index(drop=True, inplace=True)
    two.reset_index(drop=True, inplace=True)
    
    one = one.to_numpy()
    two = two.to_numpy()
    
#     print(one)
    
#     print(two)


#     if one[0][link] != two[0][link]:
#         return
#     else:
    one_fraction = one[0][views] / (one[0][views] + two[0][views])
    two_fraction = two[0][views] / (one[0][views] + two[0][views])

    one[0][views] = one[0][views] + two[0][views]
    one[0][unique_views] = one[0][unique_views] + two[0][unique_views]
    one[0][entrances] = one[0][entrances] + two[0][entrances]
    one[0][avg_time] = one[0][avg_time] * one_fraction + two[0][avg_time] * two_fraction
    one[0][bounce_rate] = one[0][bounce_rate] * one_fraction + two[0][bounce_rate] * two_fraction
    one[0][exit_percent] = one[0][exit_percent] * one_fraction + two[0][exit_percent] * two_fraction

    x = pd.DataFrame(one, columns=final.columns)

    return x
    

# head = combine(final.loc[[7]], final.loc[[8]])

# head
# final.loc[[7]].to_numpy()[0][link]
    

In [13]:
def combinegroup(df):
    df.reset_index(drop=True, inplace=True)
    if len(df) == 1:
        return df
    
    
    if len(df) == 2:
        return combine(df.loc[[0]], df.loc[[1]])
    
    new = combine(df.loc[[0]], df.loc[[1]])
    for i in range(2, len(df)):
        new = combine(new, df.loc[[i]])
    
    return new

test = final.loc[final['link'] == 'https://dailybruin.com/2020/06/15/ucla-announces-fall-quarter-plans-for-classes-housing-health-precautions/']
test

har = combinegroup(test)


In [14]:

links = final['link'].tolist()

def remove_duplicates(x):
  return list(dict.fromkeys(x))

links = remove_duplicates(links)

print(len(links))




748


In [15]:
aggregate = []


for i,  link in enumerate(links):
    try:
        group = final.loc[final['link'] == link]

        aggregate.append(combinegroup(group))
    except:
        print(i, link)



last = pd.concat(aggregate)
print(len(last))

748


In [16]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
}

for i, row in last.iterrows():
    if row.image_url == 'todo':
        try:
            req = Request(row.link, headers=headers)
            html = urlopen(req).read()
            soup = BeautifulSoup(html)

            images = soup.findAll('meta', {'property': 'og:image'})
            if len(images) == 1:
                image = images[0]['content']
                last.loc[i].image_url = image
                print(f'Processed row {i}, {image}')
            else:
                print(f'Can\'t find image: {row.link}')
        except:
            print(f'Error on {row.link}')

In [17]:
last.reset_index(drop=True, inplace=True)


last.to_csv("merged_duplicates_removed.csv")

In [18]:
test = last.head(10)

In [19]:
import numpy as np
data = pd.read_csv("merged_duplicates_removed.csv")

data.drop(data.columns[0], axis=1, inplace=True)

x = data.head(3)

y = x.to_numpy()

n = np.concatenate((y[0], y[1], ['HELLO']), axis=0)

n

array([0, '/', 90941.0, 74000.0, '0:01:24', 63825.0, 0.004599, 0.004502,
       '$0.00', 'Graduation Issue 2020',
       'https://gradissue2020.dailybruin.com/',
       'Fri, 12 Jun 2020 17:09:01 +0000', 'Camayak_180',
       "We are past the tipping point. Where do we go from here? Welcome to the Daily Bruin's 2020 Graduation Issue.",
       "['camayak_180']", "['interactive-page']", 108, nan, 1,
       '/2020/06/04/students-sign-petition-to-fire-ucla-lecturer-after-many-find-his-email-insensitive/',
       41497.0, 38105.0, '0:04:55', 37002.0, 0.00907, 0.008916, '$0.00',
       'Students sign petition to fire UCLA lecturer after many find his email insensitive',
       'https://dailybruin.com/2020/06/04/students-sign-petition-to-fire-ucla-lecturer-after-many-find-his-email-insensitive/',
       'Thu, 04 Jun 2020 23:47:55 +0000', 'Camayak_8ce8660ac1',
       ' This post was updated June 5 at 1:35 p.m.    Editor\'s note: This post was updated to include statements from the email Gordon

In [20]:
def joinrows(rowzero, rowone, views_col_num):
    higher_page_views = 1;
    
    if rowzero[views_col_num] >  rowone[views_col_num]:
        higher_page_views = 0
    
    joined = np.concatenate((rowzero, rowone, [higher_page_views]), axis=0)
    
    return joined

def joindataset(data):
    
    views = data.columns.get_loc("views")
    
    cols = data.columns
    cols_zero = cols + "_zero"
    cols_one = cols + "_one"

    newcols = list(cols_zero) + list(cols_one) + ['label']

    data = data.to_numpy()

    joined_data = []
    i = len(data) - 1
    while i >= 0:
        j = i - 1
        while j >= 0:
            joined_data.append(joinrows(data[i], data[j], views))
            joined_data.append(joinrows(data[j], data[i], views))

            j -= 1
        i -= 1
        
            
    joined_df = pd.DataFrame(joined_data, columns=newcols)
    
    return joined_df

joined_data= joindataset(data)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

print(len(joined_data))

558756


In [21]:
print(len(data))

748


In [22]:
joined_data.head()

,Unnamed: 0.1_zero,page_zero,views_zero,unique_views_zero,avg_time_zero,entrances_zero,bounce_rate_zero,exit_percent_zero,page_value_zero,title_zero,link_zero,pubDate_zero,creator_zero,encoded_text_zero,authors_zero,categories_zero,article_lens_zero,image_url_zero,Unnamed: 0.1_one,page_one,views_one,unique_views_one,avg_time_one,entrances_one,bounce_rate_one,exit_percent_one,page_value_one,title_one,link_one,pubDate_one,creator_one,encoded_text_one,authors_one,categories_one,article_lens_one,image_url_one,label
0,747,/2020/09/22/jacks-facts-pac-12-must-embrace-ch...,22.0,19.0,0:01:55,10.0,0.007000,0.004545,$0.00,Jack's Facts: Pac-12 must embrace change to co...,https://dailybruin.com/2020/09/22/jacks-facts-...,"Tue, 22 Sep 2020 18:05:06 +0000",Camayak_ee4369071f,Its time for change in the Pac-12. Fans have ...,['cap-jack-perez'],"['sports', 'sports-columns']",4603,NaN,746,/2020/10/21/graphic-donald-trumps-covid-19-dia...,23.0,20.0,0:02:16,11.0,0.008182,0.004783,$0.00,Graphic: Donald Trump's COVID-19 Diagnosis,https://dailybruin.com/2020/10/21/graphic-dona...,"Wed, 21 Oct 2020 18:34:56 +0000",Camayak_0570a3a4bf,,['cap-farrah-au-yeung'],['graphics'],6,NaN,1
1,746,/2020/10/21/graphic-donald-trumps-covid-19-dia...,23.0,20.0,0:02:16,11.0,0.008182,0.004783,$0.00,Graphic: Donald Trump's COVID-19 Diagnosis,https://dailybruin.com/2020/10/21/graphic-dona...,"Wed, 21 Oct 2020 18:34:56 +0000",Camayak_0570a3a4bf,,['cap-farrah-au-yeung'],['graphics'],6,NaN,747,/2020/09/22/jacks-facts-pac-12-must-embrace-ch...,22.0,19.0,0:01:55,10.0,0.007000,0.004545,$0.00,Jack's Facts: Pac-12 must embrace change to co...,https://dailybruin.com/2020/09/22/jacks-facts-...,"Tue, 22 Sep 2020 18:05:06 +0000",Camayak_ee4369071f,Its time for change in the Pac-12. Fans have ...,['cap-jack-perez'],"['sports', 'sports-columns']",4603,NaN,0
2,747,/2020/09/22/jacks-facts-pac-12-must-embrace-ch...,22.0,19.0,0:01:55,10.0,0.007000,0.004545,$0.00,Jack's Facts: Pac-12 must embrace change to co...,https://dailybruin.com/2020/09/22/jacks-facts-...,"Tue, 22 Sep 2020 18:05:06 +0000",Camayak_ee4369071f,Its time for change in the Pac-12. Fans have ...,['cap-jack-perez'],"['sports', 'sports-columns']",4603,NaN,745,/2020/06/11/editorial-cartoon-green-screen-gra...,24.0,24.0,0:00:33,3.0,0.003333,0.001250,$0.00,Editorial Cartoon: Green screen graduation,https://dailybruin.com/2020/06/11/editorial-ca...,"Thu, 11 Jun 2020 21:28:02 +0000",azhang3,&nbsp;,['cap-azhang3'],"['cartoons', 'opinion']",8,NaN,1
3,745,/2020/06/11/editorial-cartoon-green-screen-gra...,24.0,24.0,0:00:33,3.0,0.003333,0.001250,$0.00,Editorial Cartoon: Green screen graduation,https://dailybruin.com/2020/06/11/editorial-ca...,"Thu, 11 Jun 2020 21:28:02 +0000",azhang3,&nbsp;,['cap-azhang3'],"['cartoons', 'opinion']",8,NaN,747,/2020/09/22/jacks-facts-pac-12-must-embrace-ch...,22.0,19.0,0:01:55,10.0,0.007000,0.004545,$0.00,Jack's Facts: Pac-12 must embrace change to co...,https://dailybruin.com/2020/09/22/jacks-facts-...,"Tue, 22 Sep 2020 18:05:06 +0000",Camayak_ee4369071f,Its time for change in the Pac-12. Fans have ...,['cap-jack-perez'],"['sports', 'sports-columns']",4603,NaN,0
4,747,/2020/09/22/jacks-facts-pac-12-must-embrace-ch...,22.0,19.0,0:01:55,10.0,0.007000,0.004545,$0.00,Jack's Facts: Pac-12 must embrace change to co...,https://dailybruin.com/2020/09/22/jacks-facts-...,"Tue, 22 Sep 2020 18:05:06 +0000",Camayak_ee4369071f,Its time for change in the Pac-12. Fans have ...,['cap-jack-perez'],"['sports', 'sports-columns']",4603,NaN,744,/2020/10/24/students-and-faculty-make-calls-to...,26.0,25.0,0:01:30,3.0,0.006667,0.003462,$0.00,Students and Faculty Make Calls to Divest UCPD...,https://dailybruin.com/2020/10/24/students-and...,"Sat, 24 Oct 2020 20:17:07 +0000",Camayak_43d459acbe,"[embed url=""https://youtu.be/1mBpkD1YSJw"" con...",['cap-sophia-king'],['video'],397,NaN,1


In [ ]:
joined_data.to_csv("comparison_dataset.csv")